# ARTIC pipeline example

This is a brief run through of the ARTIC pipeline. It covers:

* installing the pipeline
* what files you need
* what commands to use
* what output is produced

***

## Using this notebook

We are using a [jupyter notebook](https://jupyter.org/) for this example so that we can host it on [Binder](mybinder.org). If you want to run the commands for yourself on the command line, you will need to remove the leading `!` which is before all the code in this notebook (which is telling jupyter to execute a system command).

To run this notebook, you can click on each cell and press Run. Be sure to wait for the cell to complete before moving on to the next one. It might take a minute or so for each cell to complete.

## Installing the pipeline

If you are using this notebook via Binder, or the GitHub repository, the pipeline is already installed. For reference though, you can install the pipeline from conda:

```
conda create -n artic-env
conda activate artic-env
conda install -c bioconda -c conda-forge artic
```

We are using version 1.2.1, check you are on the same:

In [2]:
!artic -v

artic 1.2.1


## Data

To begin, we need some data. This repository already has some data for you to use, which was generated from a SARS-COV-2 positive control sample at the University of Birmingham. If you want to obtain the data for yourself, you can run the following:

```
wget http://artic.s3.climb.ac.uk/BHAM-Run88-PTC.fastq
```

This test data is only the FASTQ reads from the positive control sample. We have already basecalled, demuxed and filtered them from the original FAST5 data for this sample.

> Because we only have FASTQ data, this example will use the **medaka** workflow of the ARTIC pipeline. This is because the **nanopolish** version requires FAST5 data as well as FASTQ.


To run the **medaka** workflow ARTIC pipeline on this data, we need to know:

* what version of the ARTIC primer scheme was used
  * version 3
* what [medaka model](https://github.com/nanoporetech/medaka#models) to use
  * use `{pore}_{device}_{caller variant}_{caller version}`
  * r941_min_high_g351

As well as the FASTQ reads, we will also need:

* primer scheme (BED format)
* reference sequence (FASTA format)

### Primer scheme and reference sequence

Although the ARTIC pipeline will download these for us, we can also get them for ourselves in order to familiarise ourselves with them:

In [3]:
!artic-tools get_scheme --schemeVersion 3 scov2

[21:41:48] [artic-tools::get_scheme] starting primer scheme downloader
[21:41:48] [artic-tools::get_scheme] 	requested scheme:	scov2
[21:41:48] [artic-tools::get_scheme] 	requested version:	3
[21:41:48] [artic-tools::get_scheme] fetching manifest file
[21:41:48] [artic-tools::get_scheme] 	ARTIC manifest URL:	https://raw.githubusercontent.com/artic-network/primer-schemes/master/schemes_manifest.json
[21:41:50] [artic-tools::get_scheme] 	ARTIC repository DOI:	10.5281/zenodo.4004423
[21:41:50] [artic-tools::get_scheme] finding primer scheme
[21:41:50] [artic-tools::get_scheme] 	found requested scheme:	sars-cov-2 (using alias scov2)
[21:41:50] [artic-tools::get_scheme] downloading primer scheme
[21:41:51] [artic-tools::get_scheme] 	saving primers to:	scov2.v3.primer.bed
[21:41:51] [artic-tools::get_scheme] 	saving reference to:	scov2.v3.reference.fasta
[21:41:51] [artic-tools::get_scheme] comparing checksums
[21:41:51] [artic-tools::get_scheme] 	sha256 for primers:	6e98d7d5d1c6edac8ef0bac7

This will have downloaded the primer scheme (`scov2.v3.primer.bed`) and the reference sequence (`scov2.v3.reference.fasta`). You can get some information on the primer scheme using artic-tools:

In [4]:
!artic-tools validate_scheme scov2.v3.primer.bed

[21:41:56] [artic-tools::validate_scheme] starting primer scheme validator
[21:41:56] [artic-tools::validate_scheme] reading scheme
[21:41:56] [artic-tools::validate_scheme] collecting scheme stats
[21:41:56] [artic-tools::validate_scheme] 	primer scheme file:	scov2.v3.primer.bed
[21:41:56] [artic-tools::validate_scheme] 	reference sequence:	MN908947.3
[21:41:56] [artic-tools::validate_scheme] 	number of pools:	2
[21:41:56] [artic-tools::validate_scheme] 	number of primers:	218 (includes 22 alts)
[21:41:56] [artic-tools::validate_scheme] 	minimum primer size:	22
[21:41:56] [artic-tools::validate_scheme] 	maximum primer size:	57
[21:41:56] [artic-tools::validate_scheme] 	number of amplicons:	98
[21:41:56] [artic-tools::validate_scheme] 	mean amplicon size:	343
[21:41:56] [artic-tools::validate_scheme] 	maximum amplicon size:	375
[21:41:56] [artic-tools::validate_scheme] 	scheme ref. span:	30-29866
[21:41:56] [artic-tools::validate_scheme] 	scheme overlaps:	12.850247%


The primer scheme file is in a BED format, where the columns equate to the following:


| column | name       | type         | description                                               |
| :----- | :--------- | :----------- | :-------------------------------------------------------- |
| 1      | chrom      | string       | primer reference sequence                                 |
| 2      | chromStart | int          | starting position of the primer in the reference sequence |
| 3      | chomEnd    | int          | ending position of the primer in the reference sequence   |
| 4      | name       | string       | primer name                                               |
| 5      | primerPool | int          | primer pool<sup>*</sup>                                  |
| 6      | strand     | string (+/-) | primer direction                                          |

<sup>*</sup> column 5 in the BED spec is an int for score, whereas here we are using it to denote primerPool.

If you want to look at the primer scheme file, we can do that here with some Python:

In [5]:
with open("scov2.v3.primer.bed", 'r') as f:
    print(f.read())

MN908947.3	30	54	nCoV-2019_1_LEFT	1	+
MN908947.3	385	410	nCoV-2019_1_RIGHT	1	-
MN908947.3	320	342	nCoV-2019_2_LEFT	2	+
MN908947.3	704	726	nCoV-2019_2_RIGHT	2	-
MN908947.3	642	664	nCoV-2019_3_LEFT	1	+
MN908947.3	1004	1028	nCoV-2019_3_RIGHT	1	-
MN908947.3	943	965	nCoV-2019_4_LEFT	2	+
MN908947.3	1312	1337	nCoV-2019_4_RIGHT	2	-
MN908947.3	1242	1264	nCoV-2019_5_LEFT	1	+
MN908947.3	1623	1651	nCoV-2019_5_RIGHT	1	-
MN908947.3	1573	1595	nCoV-2019_6_LEFT	2	+
MN908947.3	1942	1964	nCoV-2019_6_RIGHT	2	-
MN908947.3	1875	1897	nCoV-2019_7_LEFT	1	+
MN908947.3	1868	1890	nCoV-2019_7_LEFT_alt0	1	+
MN908947.3	2247	2269	nCoV-2019_7_RIGHT	1	-
MN908947.3	2242	2264	nCoV-2019_7_RIGHT_alt5	1	-
MN908947.3	2181	2205	nCoV-2019_8_LEFT	2	+
MN908947.3	2568	2592	nCoV-2019_8_RIGHT	2	-
MN908947.3	2505	2529	nCoV-2019_9_LEFT	1	+
MN908947.3	2504	2528	nCoV-2019_9_LEFT_alt4	1	+
MN908947.3	2882	2904	nCoV-2019_9_RIGHT	1	-
MN908947.3	2880	2902	nCoV-2019_9_RIGHT_alt2	1	-
MN908947.3	2826	2850	nCoV-2019_10_LEFT	2	+
MN908947.3	3183	

## Running the pipeline

Now we have the primer scheme, reference sequence and our FASTQ data. We can run the pipeline!

In [6]:
!artic minion \
    --normalise 100 \
    --threads 2 \
    --medaka \
    --medaka-model r941_min_high_g351 \
    --strict \
    --read-file ../data/BHAM-Run88-PTC.fastq.gz \
    scov2/V3 \
    my_example

could not find primer scheme and reference sequence, attempting to download
Running:  artic-tools get_scheme scov2 --schemeVersion 3
[21:42:18] [artic-tools::get_scheme] starting primer scheme downloader
[21:42:18] [artic-tools::get_scheme] 	requested scheme:	scov2
[21:42:18] [artic-tools::get_scheme] 	requested version:	3
[21:42:18] [artic-tools::get_scheme] fetching manifest file
[21:42:18] [artic-tools::get_scheme] 	ARTIC manifest URL:	https://raw.githubusercontent.com/artic-network/primer-schemes/master/schemes_manifest.json
[21:42:19] [artic-tools::get_scheme] 	ARTIC repository DOI:	10.5281/zenodo.4004423
[21:42:19] [artic-tools::get_scheme] finding primer scheme
[21:42:19] [artic-tools::get_scheme] 	found requested scheme:	sars-cov-2 (using alias scov2)
[21:42:19] [artic-tools::get_scheme] downloading primer scheme
[21:42:19] [artic-tools::get_scheme] 	saving primers to:	scov2.v3.primer.bed
[21:42:19] [artic-tools::get_scheme] 	saving reference to:	scov2.v3.reference.fasta
[21:42

[21:43:17 - PWorker] All done, 15 remainder regions.
[21:43:17 - Predict] Processing 15 short region(s).
[21:43:17 - ModelLoad] Building model with cudnn optimization: False
[21:43:18 - DLoader] Initializing data loader
[21:43:18 - PWorker] Running inference for 0.0M draft bases.
[21:43:18 - Sampler] Initializing sampler for consensus of region MN908947.3:2504-2904.
[21:43:18 - Feature] Processed MN908947.3:2504.0-2903.0 (median depth 177.0)
[21:43:18 - Sampler] Took 0.01s to make features.
[21:43:19 - PWorker] All done, 0 remainder regions.
[21:43:19 - DLoader] Initializing data loader
[21:43:19 - Sampler] Initializing sampler for consensus of region MN908947.3:3771-4164.
[21:43:19 - PWorker] Running inference for 0.0M draft bases.
[21:43:19 - Feature] Processed MN908947.3:3771.0-4163.0 (median depth 192.0)
[21:43:19 - Sampler] Took 0.02s to make features.
[21:43:20 - PWorker] All done, 0 remainder regions.
[21:43:20 - DLoader] Initializing data loader
[21:43:20 - Sampler] Initializin

[21:43:33 - TrimOverlap] MN908947.3:9229.1-9584.0 and MN908947.3:9784.0-10152.1 cannot be concatenated as there is no overlap and they do not abut.
[21:43:33 - TrimOverlap] MN908947.3:9801.0-10170.0 and MN908947.3:10362.0-10750.3 cannot be concatenated as there is no overlap and they do not abut.
[21:43:33 - TrimOverlap] MN908947.3:10374.0-10762.0 and MN908947.3:10999.0-11393.0 cannot be concatenated as there is no overlap and they do not abut.
[21:43:33 - TrimOverlap] MN908947.3:10999.0-11393.0 and MN908947.3:11555.0-11925.0 cannot be concatenated as there is no overlap and they do not abut.
[21:43:33 - TrimOverlap] MN908947.3:11576.0-11948.0 and MN908947.3:12110.0-12489.0 cannot be concatenated as there is no overlap and they do not abut.
[21:43:33 - TrimOverlap] MN908947.3:12110.0-12489.0 and MN908947.3:12710.0-13095.0 cannot be concatenated as there is no overlap and they do not abut.
[21:43:33 - TrimOverlap] MN908947.3:12710.0-13095.0 and MN908947.3:13307.0-13670.1 cannot be conca

[21:43:39 - Feature] Pileup counts do not span requested region, requested MN908947.3:0-29903, received 27784-28171.
[21:43:39 - Feature] Processed MN908947.3:27784.0-28171.0 (median depth 200.0)
[21:43:39 - Feature] Pileup counts do not span requested region, requested MN908947.3:0-29903, received 28394-28778.
[21:43:39 - Feature] Processed MN908947.3:28394.0-28778.0 (median depth 200.0)
[21:43:39 - Feature] Pileup counts do not span requested region, requested MN908947.3:0-29903, received 28985-29377.
[21:43:39 - Feature] Processed MN908947.3:28985.0-29377.0 (median depth 200.0)
[21:43:39 - Feature] Pileup counts do not span requested region, requested MN908947.3:0-29903, received 29486-29865.
[21:43:39 - Feature] Processed MN908947.3:29486.0-29865.0 (median depth 200.0)
[21:43:39 - Sampler] Took 0.14s to make features.
[21:43:39 - Sampler] Region MN908947.3:4044.0-4449.0 (794 positions) is smaller than inference chunk length 800, quarantining.
[21:43:39 - Sampler] Region MN908947.3:

[21:43:53 - TrimOverlap] MN908947.3:8291.0-8660.0 and MN908947.3:8888.0-9263.2 cannot be concatenated as there is no overlap and they do not abut.
[21:43:53 - TrimOverlap] MN908947.3:8893.3-9270.0 and MN908947.3:9477.0-9794.0 cannot be concatenated as there is no overlap and they do not abut.
[21:43:53 - TrimOverlap] MN908947.3:9540.1-9857.0 and MN908947.3:10076.0-10449.2 cannot be concatenated as there is no overlap and they do not abut.
[21:43:53 - TrimOverlap] MN908947.3:10083.0-10458.0 and MN908947.3:10666.0-11056.2 cannot be concatenated as there is no overlap and they do not abut.
[21:43:53 - TrimOverlap] MN908947.3:10679.1-11073.0 and MN908947.3:11306.0-11672.1 cannot be concatenated as there is no overlap and they do not abut.
[21:43:53 - TrimOverlap] MN908947.3:11325.2-11692.0 and MN908947.3:11863.0-12246.1 cannot be concatenated as there is no overlap and they do not abut.
[21:43:53 - TrimOverlap] MN908947.3:11872.3-12255.0 and MN908947.3:12417.0-12801.0 cannot be concatenate

2021-01-14 21:43:56    10% of variants processed...
2021-01-14 21:43:56    20% of variants processed...
2021-01-14 21:43:56    30% of variants processed...
2021-01-14 21:43:56    40% of variants processed...
2021-01-14 21:43:56    50% of variants processed...
2021-01-14 21:43:57    60% of variants processed...
2021-01-14 21:43:57    70% of variants processed...
2021-01-14 21:43:57    80% of variants processed...
2021-01-14 21:43:57    100% of variants processed.
2021-01-14 21:43:57 Calling initial genotypes using pair-HMM realignment...
Running: artic_vcf_filter --medaka my_example.merged.vcf my_example.pass.vcf my_example.fail.vcf
Running: bgzip -f my_example.pass.vcf
Running: tabix -p vcf my_example.pass.vcf.gz
Running: artic_make_depth_mask --store-rg-depths ./primer-schemes/scov2/V3/scov2.reference.fasta my_example.primertrimmed.rg.sorted.bam my_example.coverage_mask.txt
Running: artic_mask ./primer-schemes/scov2/V3/scov2.reference.fasta my_example.coverage_mask.txt my_example.fail

That's it! Let's have a quick run through of the parameters we used so that we can understand what was happening.

|parameter|explanation|
|:--------|:----------|
|`normalise`| This caps amplicon coverage to 200 reads, used mainly to speed up the pipeline run. |
|`threads`| This sets the number of CPU threads to use during the pipeline. We set this to 2 here as that is the limit on Binder, but if you are playing along at home you can increase this to make things run a bit more quickly. |
|`medaka`| This tells the ARTIC pipeline to use the **medaka** workflow|
|`medaka-model`| This specifies which model to use for the **medaka** program calls.|
|`strict`| This runs an additional filtering of reported variants, checking them in overlap regions of the primer scheme to see if they are artifacts reported in only one primer pool.|
|`read-file`| This tells the pipeline where to find the reads.|
|`scov/V3`| This specifies the name of the primer scheme and the version to use. If it isn't found locally, the pipeline will try finding it in the ARTIC primer scheme repository.|
|`my_exmple`| The name to give this pipeline run, all output will have this prepended to the filenames.|


## Pipeline output

### Files

Now it is time to check what the pipeline has produced for us. The files we are most interested in are:

|filename|description|
|:-------|:----------|
|`my_example.trimmed.rg.sorted.bam`| the post-processed alignment of reads to the reference genome. |
|`my_example.primertrimmed.rg.sorted.bam` | the post-processed alignment with additional softmasking to exclude primer sequences. |
|`my_example.vcfreport.txt` | a report evaluating reported variants against the primer scheme. |
|`my_example.pass.vcf.gz`    | detected variants that passed filters. |
|`my_example.consensus.fasta`  | the consensus sequence for the input sample. |
|`my_example..muscle.out.fasta` | an alignment of the consensus sequence against the reference sequence. |

### QC report

Before we look at these files, we can use MultiQC to check our amplicon coverage (pre-normalisation to 100) and if we have any variants reported that may be a result of contamination. Use the ARTIC MultiQC plugin:

In [7]:
!multiqc .

[INFO   ]         multiqc : This is MultiQC v1.9
[INFO   ]         multiqc : Template    : default
[INFO   ]         multiqc : Searching   : /Users/willrowe/Desktop/artic-pipeline-example/notebooks
Searching 37 files..  [####################################]  100%          
[INFO   ]  custom_content : custom_data_lineplot: Found 1 samples (linegraph)
[INFO   ]  custom_content : custom_data_json_table: Found 1 samples (table)
[INFO   ]         multiqc : Compressing plot data
[INFO   ]         multiqc : Report      : multiqc_report.html
[INFO   ]         multiqc : Data        : multiqc_data
[INFO   ]         multiqc : MultiQC complete


This will have produced a report HTML file called [multiqc_report.html](multiqc_report.html). You can click on that link or use the following code to view it in this notebook:

In [8]:
from IPython.display import IFrame
IFrame(src='./multiqc_report.html', width=600, height=400)

Use the report to see if any amplicons are marked as low coverage due to insufficient reads being assigned. Also use the report to see if there are any overlap variant fails. This is when a variant is idenified within an amplicon overlap region of the scheme but is found in only one amplicon.

### VCF files

As mentioned above, the file containing the filtered variants is `my_example.pass.vcf.gz`. Let's take a look:

In [9]:
# import pyVCF and open the variant file
import vcf
vcf_reader = vcf.Reader(filename="my_example.pass.vcf.gz")

# print the variants
print("chromosome\tpos\tref\talt")
for record in vcf_reader:
    print("{}\t{}\t{}\t{}" .format(record.CHROM, record.POS, record.REF, record.ALT))

chromosome	pos	ref	alt
MN908947.3	2618	A	[G]
MN908947.3	8782	C	[T]
MN908947.3	18488	T	[C]
MN908947.3	21846	C	[T]
MN908947.3	23605	T	[G]
MN908947.3	26354	T	[A]
MN908947.3	28144	T	[C]
MN908947.3	29366	C	[T]
MN908947.3	29596	A	[G]


### FASTA files

We also have the consensus sequence and the alignment files to look out. Here is an example of how we can look at them with Python:

In [10]:
from Bio import AlignIO
alignment = AlignIO.read("my_example.muscle.out.fasta", "fasta")
print("alignment length= {}\n" .format(alignment.get_alignment_length()))
for record in alignment:
    print("{}\n{}".format(record.id, record.seq))

alignment length= 29903

my_example/ARTIC/medaka
...
MN908947.3
...


## Questions

Now we have reached the end of the example, here are some questions for you to try. You might need to add a few more cells below so that you can use the files and Python that we used earlier to help answer them:

* how many amplicons were classed as low coverage and what was the reported coverage for these?
* how many variants were identified that were subsequently marked as FAIL?
* how many variants were incorporated into the consensus genome?
* how many Ns are in the consensus genome and why might they be there?